In [12]:
import numpy as np
import geopandas as gpd
import fiona
import pandas as pd
import rasterio
from rasterio.mask import mask
from shapely.geometry import MultiPolygon
from rasterstats import zonal_stats


data_dir = '../data'

# Load the data
tree_dataset = gpd.read_file(data_dir + '/raw_data/geodata_stadt_Zuerich/trees/data/data.gpkg')
flats_duration = gpd.read_file(data_dir + '/derived_data/flats_duration.gpkg')

# List all layers in the geopackage



In [9]:
vbz_lines = []
vbz_points = []

for layer in fiona.listlayers(data_dir + '/raw_data/geodata_stadt_Zuerich/vbz/data/data.gpkg'):
    gdf = gpd.read_file(data_dir + '/raw_data/geodata_stadt_Zuerich/vbz/data/data.gpkg', layer=layer)
    # Check first non-null geometry in the layer
    first_valid_geom = gdf.geometry[gdf.geometry.notna()].iloc[0]
    
    if first_valid_geom.geom_type == 'LineString':
        vbz_lines.append(gdf)
    elif first_valid_geom.geom_type == 'Point':
        vbz_points.append(gdf)

# Merge layers by geometry type
vbz_lines = pd.concat(vbz_lines)
vbz_points = pd.concat(vbz_points)

/home/silas/miniconda3/envs/geoenv/lib/python3.10/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


In [ ]:
import folium

# Create a base map centered on Zurich
m = folium.Map(location=[47.3769, 8.5417], zoom_start=12)

# Convert both lines and points to EPSG:4326 for folium
vbz_lines_4326 = vbz_lines.to_crs(epsg=4326)
vbz_points_4326 = vbz_points.to_crs(epsg=4326)

# Add VBZ lines
for idx, row in vbz_lines_4326.iterrows():
    if pd.notnull(row['arttext']):
        folium.GeoJson(
            row['geometry'],
            tooltip=f"Type: {row['arttext']}",
            style_function=lambda x: {'color': 'red', 'weight': 2}
        ).add_to(m)
    else:
        folium.GeoJson(
            row['geometry'], 
            style_function=lambda x: {'color': 'blue', 'weight': 1}
        ).add_to(m)

# Add VBZ points
for idx, row in vbz_points_4326.iterrows():
    if pd.notnull(row['typ']):
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=3,
            color='green',
            fill=True,
            popup=f"Type: {row['typ']}"
        ).add_to(m)
    else:
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=2,
            color='orange',
            fill=True
        ).add_to(m)

m

In [13]:


def suitability_analysis(
    buffer_dist_vbz: float,
    buffer_trees: float,
    max_slope: float,
    parking_lots: gpd.GeoDataFrame,
    slope_path: str,
    trees: gpd.GeoDataFrame,
    vbz_lines: gpd.GeoDataFrame,
    vbz_points: gpd.GeoDataFrame
):
    # Step 1: Filter parking types
    parking_filtered = parking_lots[
        ~parking_lots['parking'].isin(['underground', 'multi-storey'])
    ].copy()

    # Step 2: Process VBZ features
    vbz_line_buffers = vbz_lines.buffer(buffer_dist_vbz)
    vbz_point_buffers = vbz_points.buffer(buffer_dist_vbz)

    # Step 3: Buffer trees
    tree_buffers = trees.buffer(buffer_trees)

    # Step 4: Combine all buffers
    all_buffers = gpd.GeoSeries(
        list(vbz_line_buffers) + list(vbz_point_buffers) + list(tree_buffers),
        crs=parking_lots.crs
    ).unary_union

    # Step 5: Zonal statistics for slope
    with rasterio.open(slope_path) as src:
        slope_stats = zonal_stats(
            parking_filtered,
            slope_path,
            stats=['mean', 'max'],
            nodata=src.nodata
        )

    # Add slope stats to parking lots
    parking_filtered['slope_mean'] = [s['mean'] for s in slope_stats]
    parking_filtered['slope_max'] = [s['max'] for s in slope_stats]

    # Step 6: Filter by slope
    slope_filtered = parking_filtered[parking_filtered['slope_mean'] <= max_slope]

    # Step 7: Spatial difference with buffers
    final_areas = slope_filtered.geometry.difference(all_buffers)

    # Step 8: Calculate areas and filter
    result = gpd.GeoDataFrame(geometry=final_areas, crs=parking_lots.crs)
    result['area'] = result.geometry.area
    suitable = result[result['area'] >= 16]

    # Save output
    return suitable


In [11]:

# Example usage:
suitability_analysis(
    buffer_dist_vbz=2,
    buffer_trees=5,  # Example tree buffer distance
    max_slope=10,    # Example max slope
    parking_lots_path='parking.gpkg',
    slope_path='slope.tif',
    tree_locations_path='trees.gpkg',
    vbz_line_paths= vbz_lines
    vbz_point_paths= vbz_points
    output_path='suitable_parking.geojson'
)



DriverError: parking.gpkg: No such file or directory

In [14]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
from shapely.geometry import mapping

def suitability_analysis(
    buffer_dist_vbz: float,
    buffer_trees: float,
    max_slope: float,
    parking_lots_path: str,
    slope_path: str,
    tree_locations_path: str,
    vbz_line_paths: list[str],
    vbz_point_paths: list[str],
    output_path: str
):
    # Read input data
    parking_lots = gpd.read_file(parking_lots_path)
    trees = gpd.read_file(tree_locations_path)

    # Step 1: Filter parking types
    parking_filtered = parking_lots[
        ~parking_lots['parking'].isin(['underground', 'multi-storey'])
    ].copy()

    # Step 2: Process VBZ features
    # Merge and buffer line features
    vbz_lines = gpd.GeoDataFrame(
        pd.concat([gpd.read_file(p) for p in vbz_line_paths]),
        crs=parking_lots.crs
    )
    vbz_line_buffers = vbz_lines.buffer(buffer_dist_vbz)

    # Merge and buffer point features
    vbz_points = gpd.GeoDataFrame(
        pd.concat([gpd.read_file(p) for p in vbz_point_paths]),
        crs=parking_lots.crs
    )
    vbz_point_buffers = vbz_points.buffer(buffer_dist_vbz)

    # Step 3: Buffer trees
    tree_buffers = trees.buffer(buffer_trees)

    # Step 4: Combine all buffers
    all_buffers = gpd.GeoSeries(
        list(vbz_line_buffers) + list(vbz_point_buffers) + list(tree_buffers),
        crs=parking_lots.crs
    ).unary_union

    # Step 5: Custom zonal statistics for slope
    with rasterio.open(slope_path) as src:
        # Ensure CRS match
        parking_filtered = parking_filtered.to_crs(src.crs)
        
        slope_means = []
        slope_maxs = []
        
        for _, row in parking_filtered.iterrows():
            geom = [mapping(row.geometry)]
            
            try:
                # Extract raster values within polygon
                out_image, out_transform = mask(
                    src, geom, crop=True, nodata=src.nodata
                )
                values = out_image[0].flatten()
                valid_values = values[values != src.nodata]
                
                if valid_values.size > 0:
                    slope_means.append(np.nanmean(valid_values))
                    slope_maxs.append(np.nanmax(valid_values))
                else:
                    slope_means.append(np.nan)
                    slope_maxs.append(np.nan)
            except ValueError:
                slope_means.append(np.nan)
                slope_maxs.append(np.nan)

    # Add slope stats to parking lots
    parking_filtered['slope_mean'] = slope_means
    parking_filtered['slope_max'] = slope_maxs

    # Step 6: Filter by slope
    slope_filtered = parking_filtered[parking_filtered['slope_mean'] <= max_slope]

    # Step 7: Spatial difference with buffers
    final_areas = slope_filtered.geometry.difference(all_buffers)

    # Step 8: Calculate areas and filter
    result = gpd.GeoDataFrame(geometry=final_areas, crs=parking_lots.crs)
    result['area'] = result.geometry.area
    suitable = result[result['area'] >= 16]

    # Save output
    suitable.to_file(output_path, driver='GeoJSON')
    return suitable
